In [1]:
import numpy as np
import pandas as pd
import pylab as plt
import h5py
import matplotlib
%matplotlib inline
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import astropy.table
import subprocess as sp
import glob
import healpy as hp

In [2]:
flist = glob.glob('/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat*.fits')
#generates list of filenames in the directory above

pxnums = [] #initiates pxnums (see below)

for fname in flist:                                         #generates list of ints (pxnums)
    pxnums.append(int(fname.split('.')[1].split('_')[-1])) #containing pixel numbers
    
pxnums = np.array(pxnums) #converts pxnums to array

ra, dec = hp.pix2ang(32, pxnums, lonlat=True) #gets the right ascension and declination
                                              #of each pixel in two arrays

In [3]:
data_boss_north = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz')
#BOSS data for the northern hemisphere

data_boss_south = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_South.fits.gz')
#BOSS data for the southern hemisphere

data_boss_all = astropy.table.vstack([data_boss_north, data_boss_south])
#vertical stacking of previous two tables

pxnums_boss = hp.ang2pix(32, data_boss_all['RA'], data_boss_all['DEC'], lonlat=True)
#array of pixel numbers (with repetition for objects in the same pixel!)

overlap_boss_delve = np.in1d(pxnums_boss, pxnums)
#boolean array mask for overlapping pixels in BOSS 


In [4]:
data_boss_good = data_boss_all[overlap_boss_delve]

#all the objects in BOSS that are in pixels also present in DELVE DR2


In [5]:
px_overlap = np.unique(pxnums_boss[overlap_boss_delve])

#all the pixels present in both catalogs

In [6]:
def pixelstyle(pixel):
    if pixel < 10000: return f'0{pixel}' #formats pixel such that they always have 5 digits
    else: return f'{pixel}'

In [7]:
#here we will create a single Astropy Table of the first 800 pixels

dr2_all = []
for pixel in pxnums[5000:5500]: #parallelization
    dr2_i = astropy.table.Table.read(f'/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat_hpx_{pixelstyle(pixel)}.fits')
    bandlist = ['G', 'R', 'I', 'Z']
    for band in bandlist:
        dr2_i[f'EXTENDED_CLASS_{band}'] = (1.0*((dr2_i[f'SPREAD_MODEL_{band}'] + 3*dr2_i[f'SPREADERR_MODEL_{band}']) > 0.005) 
        + 1.0*((dr2_i[f'SPREAD_MODEL_{band}'] + dr2_i[f'SPREADERR_MODEL_{band}']) > 0.003) 
        + 1.0*((dr2_i[f'SPREAD_MODEL_{band}'] - dr2_i[f'SPREADERR_MODEL_{band}']) > 0.003))
    dr2_i_f = dr2_i[dr2_i['EXTENDED_CLASS_G']>1] #filtering out likely stars
    dr2_all.append(dr2_i_f)
dr2_all = astropy.table.vstack(dr2_all)

In [8]:
data=dr2_all

In [9]:
#here we will catalog match the first 800 pixels
#from DELVE DR2 to all of BOSS, then run the matched 
#catalog through BPZ and then callibrate

from astropy.coordinates import SkyCoord
from astropy import units as u
boss_cd = SkyCoord(ra=data_boss_good['RA']*u.degree, dec=data_boss_good['DEC']*u.degree)
delve_cd = SkyCoord(ra=data['RA']*u.deg, dec=data['DEC']*u.deg)
idx, d2d, d3d = boss_cd.match_to_catalog_sky(delve_cd)

In [10]:
good_matches = d2d < 1.0*u.arcsec
print(len(np.unique(idx[good_matches])), np.count_nonzero(good_matches))

73991 73991


In [11]:
idx_, d2d_, d3d_ = delve_cd.match_to_catalog_sky(boss_cd)

In [12]:
good_matches_ = d2d_ < 1.0*u.arcsec
print(len(np.unique(idx_[good_matches_])), np.count_nonzero(good_matches_))

73991 75515


In [13]:
data_all = astropy.table.hstack([data_boss_good[good_matches], data[idx][good_matches]])

In [14]:
data_all.write('/data/des90.a/data/raulteix/data/BOSSDELVE/BOSSDELVE11.fits', format='fits')

OSError: File /data/des90.a/data/raulteix/data/BOSSDELVE/BOSSDELVE11.fits already exists. If you mean to replace it then use the argument "overwrite=True".